In [1]:
# syft absolute
import syft as sy
from syft.types.syft_object import SyftObject
from syft import ActionObject
# syft absolute
from syft.service.user.user import UserCreate
from syft.node.credentials import SyftVerifyKey
from syft.service.action.action_permissions import ActionPermission, ActionObjectPermission

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft


In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)
node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=2050ad12e6e34c4b8ac36353c84c5a0e in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=0b8fd419b30a4e499a6e4ec8813a8fb6 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
client_low.register(email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw")
client_low_ds = node_low.login(email="newuser@openmined.org", password="pw")

Logged into <test_l: Low side Domain> as <newuser@openmined.org>


# create datasets

In [5]:
# third party
import numpy as np

In [6]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

  0%|                                                                                                                 | 0/1 [00:00<?, ?it/s]

Uploading: numpy-data


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.93it/s]


SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [7]:
mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
# private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    id=dataset_high.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=ActionObject.empty(data_node_id=client_high.id),
            shape=mock_low.shape,
            mock_is_real=True,
        )
    ],
)

res = client_low.upload_dataset(dataset_low)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  9.59it/s]


Uploading: numpy-data


In [8]:
data_low = client_low_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    return data.mean()


compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/ef573e02b13c4de8ab935d223b35c56b.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=36c199461b994117b984ece31b94ec62 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_756: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Approving request for domain ephemeral_node_compute_mean_756
af1f75115c4245a78579a39f715ba9f5
override True
SyftInfo: Landing the ephmeral node...


```python
Pointer
```
2.0

In [9]:
client_low_ds.code.request_code_execution(compute_mean)

syft.service.request.request.Request

In [10]:
request = client_low.requests[0]

In [11]:
code = client_low.code[0]

TEO HERE

In [12]:
# request.changes

In [13]:
items_to_sync = [request, code]

- What do we do with properties here?
  - just set them to None

In [14]:
def get_stash_for_item(item, node):
    services = list(node.python_node.service_path_map.values())

    all_stashes = dict()
    for serv in services:
        if (_stash := getattr(serv, "stash", None)) is not None:
            all_stashes[_stash.object_type] = _stash

    stash = all_stashes.get(type(item), None)
    return stash

In [15]:
# job.node_uid

In [16]:
node_high.python_node.id

<UID: 8212e6797fde4c3fba4fc53ab555a886>

In [17]:
def set_obj_ids(x, node_uid, verify_key):
    if hasattr(x, "__dict__") and isinstance(x, SyftObject):
        for prop, val in x.__dict__.items():
            if isinstance(val, (list, tuple)):
                for v in val:
                    set_obj_ids(v, node_uid, verify_key)
            elif isinstance(val, dict):
                for v in val.values():
                    set_obj_ids(v, node_uid, verify_key)
            else:
                set_obj_ids(val, node_uid, verify_key)
        x.syft_node_location=node_uid
        x.syft_client_verify_key=verify_key
        if hasattr(x, "node_uid"):
            x.node_uid=node_uid
                    
                    
    # TODO: hadnle nested collections
            

In [18]:
# add_permissions_for_actionobject(job_high.result, node_low, node_high)

In [19]:
def add_permissions_for_actionobject(obj, node_to, node_from):
    _id = obj.id.id 
    all_permissions = node_from.python_node.get_service("actionservice").store.permissions[_id]
    read_permissions = [x for x in all_permissions if "READ" in x]

    store_to = node_to.python_node.get_service("actionservice").store
    for read_permission in read_permissions:
        creds, perm_str = read_permission.split("_") 
        perm = ActionPermission[perm_str]
        permission = ActionObjectPermission(uid=_id, permission=perm, credentials=SyftVerifyKey(creds))
        res = store_to.add_permission(permission)
        print("Created permission", res)

In [20]:
def create_actionobject(action_object, client, node_to, node_from):
    action_object = action_object.refresh_object()
    res = client.api.services.action.set(action_object)
    print("Created", res)

    if node_to.python_node.node_side_type.value == "low":
        add_permissions_for_actionobject(action_object, node_to, node_from)
    


TODOS
- define datasets to point to high side
- define usercodestatus such that it points to high side
- we need to update the local request object once we call .approve(), because otherwise you will sync something old (without approval)
- admin execution on the high side does not fill the cache, so it will break low side execution becaeuse there is nothing in the cache. Unless you do accept_by_depositing_result
- if you dont do accept_by_depositing_result with an actionobject, but with a python object
- we have a logic in accept_by_depositing_result that if you pass in an actionobject, while you ran a job (and you are also syncing the job), the job wont be made available on the low side because it lacks permissions

In [21]:
from syft.client.api import NodeIdentity
from syft.service.code.user_code import UserCode

In [22]:
def transform_item(item, node):
    identity = NodeIdentity.from_node(node.python_node)
    if isinstance(item, UserCode):
        res = {}
        for key, val in item.status.status_dict.items():
            # todo, check if they are actually only two nodes
            res[identity] = code.status.status_dict[key]
        item.status.status_dict=res
    return item

In [23]:
def sync_items(items, node_to, node_from, client):
    for item in items_to_sync:
        
        if isinstance(item, ActionObject):
            create_actionobject(item, client, node_to, node_from)

        else:
            item = transform_item(item, node_to)
            stash = get_stash_for_item(item, node_to)
            creds=client.verify_key

            set_obj_ids(item, client.id, creds)
            exists = stash.get_by_uid(creds, item.id).ok() is not None
            if exists:
                res = stash.update(creds, item)
            else:
#                 res = stash.delete_by_uid(node.python_node.verify_key, item.id)
                res = stash.set(creds, item)
            if not res.is_ok():
                raise ValueError("")
            else:
                print("created", item)
    client._fetch_api(client.credentials)

In [24]:
sync_items(items_to_sync, node_high, node_low, client_high)

created syft.service.request.request.Request
created syft.service.code.user_code.UserCode


# Run code high and sync back result

In [25]:
data_high = client_high.datasets[0].assets[0]

In [26]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)
display(job_high)

SyftInfo: Node Landed!


```python
class Job:
    id: UID = 946c00bf9e75431899c40c067d0a2ca7
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [27]:
# wait for the result
job_high.wait().get()

5b21980419734ec5a131ae5d7243dd15
override True


17.0

In [28]:
job_info = job_high.info(public_metadata=True, result=True)

In [29]:
request = client_high.requests[0]
result_obj = job_high.result

print("result obj", result_obj.id)
request.accept_by_depositing_result(job_info)

# request.approve()
# TODO: FIX THIS
request = client_high.requests[0]
code = request.code
log = job_high._get_log_objs()

result obj 64338b7848504dc393d5160d9dadf265
Approving request for domain test_h
Approving request for domain test_h
ADDING PERMISSION [READ: ae8248c08fdf4a8c930c2c1bbcb9e178 as 9897b007f8c00ac78e0e435e3ca0be581a96b5b97e1ec425fdf73fd7e3e1bcd4] ae8248c08fdf4a8c930c2c1bbcb9e178
returning existing job
setting permission
None
None


In [30]:
client_low_ds.verify_key

9897b007f8c00ac78e0e435e3ca0be581a96b5b97e1ec425fdf73fd7e3e1bcd4

In [31]:
list(node_high.python_node.get_service("jobservice").stash.partition.permissions.values())[0]

{'9897b007f8c00ac78e0e435e3ca0be581a96b5b97e1ec425fdf73fd7e3e1bcd4_READ',
 'd311a667006cbb56614a062b2bbc7a733b5a4a8edd5293e0f34bb5c75f51277d_EXECUTE',
 'd311a667006cbb56614a062b2bbc7a733b5a4a8edd5293e0f34bb5c75f51277d_OWNER',
 'd311a667006cbb56614a062b2bbc7a733b5a4a8edd5293e0f34bb5c75f51277d_READ',
 'd311a667006cbb56614a062b2bbc7a733b5a4a8edd5293e0f34bb5c75f51277d_WRITE'}

## Sync back to low

In [32]:
items_to_sync = [code, job_high.result, job_high, log]

In [33]:
# res = client_low.api.services.action.set(job_high.result)


In [34]:
# res

In [35]:
# type(res)

In [36]:
sync_items(items_to_sync, node_low, node_high, client_low)

created syft.service.code.user_code.UserCode
Created 17.0
Created permission None
created syft.service.job.job_stash.Job
created syft.service.log.log.SyftLog


# Run code low

- if you are a low side node
- and you are executing on an object that actually lives somewhere else
- and you have no permissions normally to do this
- and you are executing against something that has a filled cache (this means that this was filled by non mock execution

In [37]:
client_low_ds.verify_key

9897b007f8c00ac78e0e435e3ca0be581a96b5b97e1ec425fdf73fd7e3e1bcd4

In [38]:
job_high.result.id.id

<UID: 64338b7848504dc393d5160d9dadf265>

In [39]:
node_high.python_node.get_service("actionservice").store.permissions[job_high.result.id.id]

{'d311a667006cbb56614a062b2bbc7a733b5a4a8edd5293e0f34bb5c75f51277d_EXECUTE',
 'd311a667006cbb56614a062b2bbc7a733b5a4a8edd5293e0f34bb5c75f51277d_OWNER',
 'd311a667006cbb56614a062b2bbc7a733b5a4a8edd5293e0f34bb5c75f51277d_READ',
 'd311a667006cbb56614a062b2bbc7a733b5a4a8edd5293e0f34bb5c75f51277d_WRITE'}

In [40]:
node_low.python_node.get_service("actionservice").store.permissions[job_high.result.id.id]

{'1c0590b894d19530970da6bf09bb302afa3e473a4cccdacbace0e20223ee3367_EXECUTE',
 '1c0590b894d19530970da6bf09bb302afa3e473a4cccdacbace0e20223ee3367_OWNER',
 '1c0590b894d19530970da6bf09bb302afa3e473a4cccdacbace0e20223ee3367_READ',
 '1c0590b894d19530970da6bf09bb302afa3e473a4cccdacbace0e20223ee3367_WRITE',
 'd311a667006cbb56614a062b2bbc7a733b5a4a8edd5293e0f34bb5c75f51277d_READ'}

In [41]:
# policy = list(node_low.python_node.get_service("usercodeservice").stash.partition.data.values())[0].output_policy

# # policy.valid

In [42]:
# policy.last_output_ids[0]

In [43]:
# policy.last_output_ids[0].id

In [44]:
# job_high.result.id

In [45]:
# code.output_policy.output_history

## Run

In [46]:
res_low = client_low_ds.code.compute_mean(data=data_low)

5b21980419734ec5a131ae5d7243dd15
override False
result Permission: [READ: ae8248c08fdf4a8c930c2c1bbcb9e178 as 9897b007f8c00ac78e0e435e3ca0be581a96b5b97e1ec425fdf73fd7e3e1bcd4] denied


In [47]:
req = client_low.requests[0]

In [49]:
res

SyftSuccess: Dataset uploaded to 'test_l'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [52]:
# res_low

In [50]:
res_low.get()

AttributeError: 'SyftError' object has no attribute 'get'

In [ ]:
# print(res_low.message)

In [53]:
node_high.python_node.dev_mode

True

In [54]:
# we need to sync the usercode itself


In [55]:
# node_low.python_node.get_service("usercodeservice").stash.partition.permissions

In [56]:
client_low_ds.verify_key

9897b007f8c00ac78e0e435e3ca0be581a96b5b97e1ec425fdf73fd7e3e1bcd4

In [57]:
res_low.get()

AttributeError: 'SyftError' object has no attribute 'get'

In [58]:
private_high.mean()

17.0

In [59]:
code_id = list(node_low.python_node.get_service("usercodeservice").stash.partition.data.values())[0].id

In [60]:
job = next(iter(node_low.python_node.get_service("jobservice").stash.partition.data.values()))

In [61]:
job.id

<UID: 946c00bf9e75431899c40c067d0a2ca7>

In [62]:
node_low.python_node.get_service("jobservice").stash.get_by_uid(client_low_ds.verify_key, job.id)

Ok(None)

In [63]:
node_low.python_node.get_service("jobservice").stash.get_by_user_code_id(client_low_ds.verify_key, code_id)

Ok([])

In [64]:
job_low = client_low_ds.code.compute_mean(data=data_low, blocking=False)

5b21980419734ec5a131ae5d7243dd15


In [65]:
job_low

SyftError: Please wait for the admin to allow the execution of this code

In [66]:
job_low.wait().get()

AttributeError: 'SyftError' object has no attribute 'wait'